In [1]:
import json
import ee
import numpy as np
import time
import subprocess
import sys
import os
import gdal
import matplotlib.pyplot as plt

# initialize ee
ee.Initialize()

%matplotlib inline

### Key User Input
Here is the kernel where we include key parameters which define our downloads. These are the following:<br>
 - **shapefile_name**: This is the path to the shapefile which defines the spatial area we are interested in.<br>
 - **key**: This is the extension that all files will have attached to them to identify the files.<br>
 - **year**: The year of data we want to download. We download only a year to keep the data volumes low. <br>
 - **download_directory**: Where we want to place the downloaded data.  

In [2]:
shapefile_name = '/media/DataShare/Alex/un_surinam/eo4sgs_inital_aoi.geojson'
key = 'test_download'
year = 2018
download_directory = '.'

We will be using the Google Earth Engine to to download the data. This is a huge repository of petabytes of data where we can cleanly aquire data for any given region within the timeframe available from each sensor. This is done in the python API by constructing a call to their API, filtering the dataset to the area and time we want and then we can export this 'Image collection' to a google 'asset'. To view all the available dataset available with Google Earth Engine, see this link: https://developers.google.com/earth-engine/datasets/catalog <br><br>
To learn more about how to use the Google Earth Engine, please refere to the following:
https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api <br><br>
Below is where we construct the call to the sensor (Landsat 8, https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2#description)

In [3]:
# Specifiy the product we are interested in. This is unique to each sensor
# and found on each the product specification page
product = 'LANDSAT/LC08/C02/T1_L2'

# setout how you want label this data. As we are using Landsat 5 we can use
# the following
dataset_custom_tag = 'LC08'

# we can open the geojson with the 'json' toolbox
geoj = json.load(open(shapefile_name, 'r'))
# we then use this json to create a Google Earth Engine Geometry Object 
geometry = ee.Geometry.Polygon(geoj['features'][0]['geometry']['coordinates'][0])

# to filter the time, we use strings with the following format
# YYY-MM-DD
start_date = '%s-01-01'%year
end_date = '%s-12-31'%year

# specificy the bands we want. These can be found here:
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2#bands
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5',
         'SR_B7']

# finally, it is good to force Google Earth Engine to use the 
# native resolution, so create a list as the same length
# as 'bands' at 30m. 
resolutions = [30,30,30,30,30,30]

# create the Image Collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date).
                 select(bands))

In [4]:
# you can interact with the immage collections with a few basic commands
# but we dont need to do anything else to the Image Collection to download
# the data. 
print ('Number of Image Aquisitions:')
print (ee_collection.size().getInfo())

Number of Image Aquisitions:
76


To download the Image Collection, we need a two step process. Firstly we need to export the datasets to a Google Drive and then we can export from the Drive to the local machine, using gsutils. This works by individually exporting all data for each band to the Drive, which is a time consuming process as the Earth Engine acually does the processing in this stage for us. We wait in the 'while' loop until this prcoess is completed. We then use 'gsutils' which is a linux command line toolbox, which we call from python. 

In [6]:
# create a list to put the tasks in so we can check their progress
tasks = []

# loop throguh each Landsat 8 band and export them to the google drive
for resolution, band in zip(resolutions, bands):
    
    # create a unique key to attach to the data     
    unique_desc = f'{key}_{band}'
    
    # USER INPUT
    # the user must use the following command to export the data
    # https://developers.google.com/earth-engine/apidocs/export-image-todrive
    
    DRIVE_NAME = '#YOUR_DRIVE_HERE#'
    
    task = ee.batch.Export.image.toDrive(
        image = ee_collection.select(band).toBands(),
        region = geometry,
        description = unique_desc,
        folder = DRIVE_NAME,
        fileNamePrefix = '%s_%s_%s_%s'%(dataset_custom_tag, key, 
                                        year, band),
        scale = resolution,
        maxPixels = 1e13)
    
    tasks.append(task)

for task in tasks:
    task.start()

# continue in this while loop until all the tasks are finished. Once
# complete they will be sitting on the google bucket

# record when we started waiting for the data
start = time.time()

while len([i.status()['state'] for i in tasks if i.status()['state'] == 'COMPLETED']) < len(bands):
    
    # use the command line 'earthengine' package to find how our 
    # downloads are going
    result = subprocess.run(['earthengine', 'task', 'list'], stdout=subprocess.PIPE)
    # get information for the most recent tasks, which is what
    # we are waiting for
    output = str(result.stdout).replace('  Export.image  ','').split('\\n')[:6]
    # format all this information
    trimmed = [[i for i in j.split(' ') if len(i) > 0] for j in output]
    prnt_lines = ['%s - %s'%(i[0],i[1]) for i in trimmed]
    second_dif = str(int(time.time() - start))+'s'
    second_dif_str_second = f"{second_dif:<5}"
    second_dif_str = '(L8 %s %s) %s'%(key, year,second_dif_str_second)
    prnt = '%s - %s // %s // %s // %s // %s // %s // %s'%(second_dif_str,prnt_lines[0][2:],prnt_lines[1],prnt_lines[2],
                                                         prnt_lines[3], prnt_lines[4], prnt_lines[5], prnt_lines[6])

    # priunt out the information so we can see how well we are doing.
    sys.stdout.write('%s\r' % (prnt,))
    sys.stdout.flush()
    # there are four possible state of the task which will be printed out:
    # READY
    # RUNNING
    # COMPLETED
    # FAILED
    

print ('\n')

# once this is done, we use gsutils to move this data from the Drive to 
# your local machine. 

# check the output directory exists and create if needs be
dest = os.path.join(download_directory)
if os.path.isdir(dest) == False:
    os.makedirs(dest)
    
# constrcut the command line call
cmd = 'gsutil mv gs://%s/%s_%s_%s* %s/'%(DRIVE_NAME, dataset_custom_tag,
                                         key, year, dest)

print ('Starting data downloads:')
# use python to call this command and move the data
subprocess.call(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)    

(L5 test_download 2018) 1220s - BWDJR4AOVVBQZYPKFH6L7ALXtest_download_SR_B7 - COMPLETED // QBVOMK3C4P6M6N7U6IKQAK4Etest_download_SR_B5 - COMPLETED // ANE7KOLIDU5YWRHW52HJ4LBAtest_download_SR_B4 - COMPLETED // ITEV6OFPQJ5T2UYCU2AN3FKUtest_download_SR_B3 - COMPLETED // UEHTK7Y3UN6KZGPXHU5ZAU66test_download_SR_B2 - COMPLETED // QQARX6J7GIYBTGQKDRMXMEZYtest_download_SR_B1 - COMPLETED // QC3S75MGXALIQBBDACWSGIFFtest_download_B12 - CANCELLED

Starting data downloads:


0